<a href="https://colab.research.google.com/github/abhishakvarshney/Hackathon---eClerx/blob/master/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
def download_data():
    !wget 'https://storage.googleapis.com/kaggle-data-sets/4133%2F8841%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1595929231&Signature=M2UzMApcy0Epe55p0OUB9uwn%2FQo0bWS09xzwITl0ctga0%2F7i%2FqAR%2BZcSOXvv%2F0EjECpRdu5%2FWiN89I9F25sg8u4bqQaC29bXUmxFoopThJscoFq5SinQzOSEIvLWNup2cEU94%2F%2BENpzDCvOYIJ2tE9yWv97HsGLf9qJFNqbsMsolIwfJZmqzXhqY%2FRmsPgjJ%2F3eG0AjF1HyNKIwRmyN3INMAhZbovEEKCSlTjmHg1L%2Bx8YuofBZXmLIOCYDDkPEv2jqP%2FTI6%2FPUQIznSvBZTRxR3e2LMyduHwSViUGUfLf59z0VvVcQaYDvGXFzJR%2F21h6PL%2BM3cO401dP6VuCG7kg%3D%3D' -O "data.zip"
    !unzip -o "data.zip"
    !rm -rf "data.zip"
download_data()

--2020-07-25 20:52:51--  https://storage.googleapis.com/kaggle-data-sets/4133%2F8841%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1595929231&Signature=M2UzMApcy0Epe55p0OUB9uwn%2FQo0bWS09xzwITl0ctga0%2F7i%2FqAR%2BZcSOXvv%2F0EjECpRdu5%2FWiN89I9F25sg8u4bqQaC29bXUmxFoopThJscoFq5SinQzOSEIvLWNup2cEU94%2F%2BENpzDCvOYIJ2tE9yWv97HsGLf9qJFNqbsMsolIwfJZmqzXhqY%2FRmsPgjJ%2F3eG0AjF1HyNKIwRmyN3INMAhZbovEEKCSlTjmHg1L%2Bx8YuofBZXmLIOCYDDkPEv2jqP%2FTI6%2FPUQIznSvBZTRxR3e2LMyduHwSViUGUfLf59z0VvVcQaYDvGXFzJR%2F21h6PL%2BM3cO401dP6VuCG7kg%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 172.253.118.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176772673 (169M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 168.58M   168MB/s    in 1.0s    

2020-07-25 

KeyboardInterrupt: ignored

In [ ]:
!pip install scattertext spacy_cld spacymoji empath

In [ ]:
# library imports
import re
import numpy as np
import pandas as pd
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import seaborn as sns
width = 0.75
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
plt.axis('off')
from nltk.corpus import stopwords
from textblob import TextBlob
import nltk
import scattertext as st
import spacy
import spacy_cld

from IPython.display import IFrame
from IPython.core.display import display, HTML
from collections import Counter
from datetime import datetime
from tqdm.notebook import tqdm as tqdm  # cool progress bars
tqdm().pandas()  # Enable tracking of progress in dataframe `apply` calls

In [ ]:
# 8192 - large enough for demonstration, larger values make network training slower
MAX_VOCAB_SIZE = 2**13
# seq2seq generally relies on fixed length message vectors - longer messages provide more info
# but result in slower training and larger networks
MAX_MESSAGE_LEN = 30  
# Embedding size for words - gives a trade off between expressivity of words and network size
EMBEDDING_SIZE = 100
# Embedding size for whole messages, same trade off as word embeddings
CONTEXT_SIZE = 100
# Larger batch sizes generally reach the average response faster, but small batch sizes are
# required for the model to learn nuanced responses.  Also, GPU memory limits max batch size.
BATCH_SIZE = 4
# Helps regularize network and prevent overfitting.
DROPOUT = 0.2
# High learning rate helps model reach average response faster, but can make it hard to 
# converge on nuanced responses
LEARNING_RATE=0.005

# Tokens needed for seq2seq
UNK = 0  # words that aren't found in the vocab
PAD = 1  # after message has finished, this fills all remaining vector positions
START = 2  # provided to the model at position 0 for every response predicted

# Implementaiton detail for allowing this to be run in Kaggle's notebook hardware
SUB_BATCH_SIZE = 1000

In [ ]:
tweets = pd.read_csv('twcs/twcs.csv',encoding='utf-8')
print(tweets.shape)
tweets.head(5)

In [ ]:
tweets.info()

In [ ]:
def missingData(data):
    '''
    @author steno
    Check the missing data in the features.
    This function returns  dataframe with the missing values.
    '''
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    md = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    md = md[md["Percent"] > 0]
    sns.set(style = 'darkgrid')
    plt.figure(figsize = (8, 4))
    plt.xticks(rotation='90')
    sns.barplot(md.index, md["Percent"],color="g",alpha=0.8)
    plt.xlabel('Features', fontsize=15)
    plt.ylabel('Percent of missing values', fontsize=15)
    plt.title('Percent missing data by feature', fontsize=15)
    return md

In [ ]:
missingData(tweets)

In [ ]:
first_inbound = tweets[pd.isnull(tweets.in_response_to_tweet_id) & tweets.inbound]

QnR = pd.merge(first_inbound, tweets, left_on='tweet_id', 
                                  right_on='in_response_to_tweet_id')

# Filter to only outbound replies (from companies)
QnR = QnR[QnR.inbound_y ^ True]
print(f'Data shape: {QnR.shape}')
QnR.head(3)

In [ ]:
missingData(QnR)

In [ ]:
# removing anonymized screen names 
def sn_replace(match):
    _sn = match.group(2).lower()
    if not _sn.isnumeric():
        # This is a company screen name
        return match.group(1) + match.group(2)
    return ''

sn_re = re.compile('(\W@|^@)([a-zA-Z0-9_]+)')
print("Removing anonymized screen names in X...")
QnR["text_x"] = QnR.text_x.progress_apply(lambda txt: sn_re.sub(sn_replace, txt))
print("Removing anonymized screen names in Y...")
QnR["text_y"] = QnR.text_y.progress_apply(lambda txt: sn_re.sub(sn_replace, txt))

In [ ]:
#making sure the dataframe contains only the needed columns
QnR = QnR[["tweet_id_x", "author_id_x","created_at_x","text_x","author_id_y","created_at_y","text_y"]]
QnR.head(5)

In [ ]:
QnR.info()

In [ ]:
count = QnR.groupby("author_id_y")["text_x"].count()
c = count[count>15000].plot(kind='barh',figsize=(10, 8), color='#619CFF', zorder=2, width=width,)
c.set_ylabel('')
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
sns.set_style('white')
sns.countplot(x='author_id_y', data=QnR)
plt.xticks(rotation = 90)
plt.title("Distributions of Number of Companies' Replies ", fontsize = 20)
plt.show()

In [ ]:
author_grouped = QnR.groupby('author_id_y')
top_support_providers = set(author_grouped.agg('count').sort_values(['tweet_id_x'], ascending=[0]).index[:20].values)

In [ ]:
QnR.loc[QnR.author_id_y.isin(top_support_providers)].groupby('author_id_y').tweet_id_x.count().sort_values().plot(kind='barh', title='Top 20 Brands by Volume')

In [ ]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

def sentiment_for(text: str) -> float:
    return sentiment_analyzer.polarity_scores(text)['compound']
QnR['inbound_sentiment'] = QnR.text_x.progress_apply(sentiment_for)

In [ ]:
QnR \
    .loc[QnR.author_id_y.isin(top_support_providers)] \
    .groupby('author_id_y') \
    .inbound_sentiment.mean() \
    .sort_values() \
    .plot(kind='barh', title='Customer Sentiment by Brand (top 20 by volume)')

In [ ]:
QnR['created_at_x'] = pd.to_datetime(QnR.created_at_x)

In [ ]:
apple_tweets = QnR \
    .loc[QnR.author_id_y == 'AppleSupport']

In [ ]:
plt.subplot(2, 1, 1)

ax = apple_tweets \
    .groupby(pd.Grouper(key='created_at_x', freq='24h')) \
    .count() \
    .tweet_id_x.sort_index() \
    .plot(title='@AppleSupport Volume & Sentiment - Impact of the iPhone X', kind='area')
ax.set_ylabel('Inbound Tweets')
    

plt.subplot(2, 1, 2)
ax = apple_tweets \
    .groupby(pd.Grouper(key='created_at_x', freq='24h')) \
    .inbound_sentiment.mean() \
    .sort_index() \
    .plot(color='red')
ax.set_ylabel('Customer Sentiment')

In [ ]:
amazonQnR = QnR[QnR["author_id_y"]=="AmazonHelp"]

In [ ]:
amazonQnR.tail(10)["text_x"]

In [ ]:
# amazonQnR["text_x"] = amazonQnR["text_x"].str.encode("utf-8")
# amazonQnR["text_x"] = amazonQnR["text_x"].apply(str)

In [ ]:
nlp_cld = spacy.load('en',disable_pipes=["tagger","ner"])
language_detector = spacy_cld.LanguageDetector()
nlp_cld.add_pipe(language_detector)

In [ ]:
doc = nlp_cld(amazonQnR.iloc[4]["text_x"])
print(doc)
print(doc._.languages)  
print(doc._.language_scores)

In [ ]:
mask = []
try:
    for i,doc in tqdm(enumerate(nlp_cld.pipe(amazonQnR["text_x"], batch_size=512))):
            if 'en' not in doc._.languages or len(doc._.languages) != 1:
                mask.append(False)
            else:
                mask.append(True)
except Exception:
    print("excepted ")

In [ ]:
amazonQnR = amazonQnR[mask]
# sample a random fraction to visually ensure that we have only English tweets
amazonQnR.sample(frac=0.0002)

In [ ]:
amazonQnR.tail(10)["text_x"]

In [ ]:
!python -m spacy download en_core_web_lg
!python -m spacy link en_core_web_lg en --force

In [ ]:
nlp = spacy.load("en",disable_pipes=["tagger"])

from spacymoji import Emoji
emoji = Emoji(nlp)
nlp.add_pipe(emoji, first=True)

In [ ]:
print(nlp.pipe_names)

In [ ]:
emojis = []
for doc in tqdm(nlp.pipe(amazonQnR["text_x"], batch_size=512)):
    if doc._.has_emoji:
        for e in doc._.emoji:
            emojis.extend(e[0])

In [ ]:
eCount = Counter(emojis)
eCount.most_common(10)

In [ ]:
response_emojis = []
for doc in tqdm(nlp.pipe(amazonQnR["text_y"], batch_size=512)):
    elist = []
    if doc._.has_emoji:
        for e in doc._.emoji:
            elist.append(e[0])
    response_emojis.append(elist)

In [ ]:
Counter([item for sublist in response_emojis for item in sublist]).most_common(10)

In [ ]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sent_analyser = SentimentIntensityAnalyzer()
positive_text = "love this phone! its the best one I've owned over the years"
negative_text = "what sort of company makes such products? this phone hangs up all the time and is totally useless"
print("positive_text sentiment : ",sent_analyser.polarity_scores(positive_text)["compound"])
print("negative_text sentiment : ",sent_analyser.polarity_scores(negative_text)["compound"])

In [ ]:
def sentiment(text):
    return (sent_analyser.polarity_scores(text)["compound"] + TextBlob(text).sentiment.polarity)/2
amazonQnR["text_x_sentiment"] = amazonQnR["text_x"].apply(sentiment)

In [ ]:
response_emojis_for_positive_queries = []
response_emojis_for_negative_queries = []
for i,sentiment in enumerate(amazonQnR["text_x_sentiment"]):
    if sentiment > 0.0:
        response_emojis_for_positive_queries.extend(response_emojis[i])
    elif sentiment < 0.0:
        response_emojis_for_negative_queries.extend(response_emojis[i])

In [ ]:
amazonQnR[amazonQnR["text_x_sentiment"]>0].head()

In [ ]:
Counter(response_emojis_for_positive_queries).most_common(10)

In [ ]:
Counter(response_emojis_for_negative_queries).most_common(10)


In [ ]:
count = QnR.groupby("author_id_y")["text_x"].count()
c = count[count>15000].plot(kind='barh',figsize=(10, 8), color='#619CFF', zorder=2, width=width,)
c.set_ylabel('')
plt.show()

In [ ]:
airlinesQnR = QnR[(QnR["author_id_y"]=="AmericanAir")|(QnR["author_id_y"]=="British_Airways")]
airlinesQnR.head(4)

In [ ]:
import nltk
nltk.download('stopwords')
airlinesQnR["text_y"] = airlinesQnR["text_y"].str.lower()  
stop = stopwords.words('english')
big_regex = re.compile(' | '.join(stop))
airlinesQnR["text_y"].progress_apply(lambda x: big_regex.sub(" ",x))

In [ ]:
import scattertext as st
nlp = spacy.load('en',disable_pipes=["tagger","ner"])
airlinesQnR['parsed'] = airlinesQnR.text_y.progress_apply(nlp)

In [ ]:
corpus = st.CorpusFromParsedDocuments(airlinesQnR,
                             category_col='author_id_y',
                             parsed_col='parsed').build()

In [ ]:
html = st.produce_scattertext_explorer(corpus,
          category='British_Airways',
          category_name='British Airways',
          not_category_name='American Airlines',
          width_in_pixels=600,
          minimum_term_frequency=10,
          term_significance = st.LogOddsRatioUninformativeDirichletPrior(),
          )

In [ ]:
filename = "americanAir-vs-britishAirways.html"
open(filename, 'wb').write(html.encode('utf-8'))
IFrame(src=filename, width = 800, height=700)

In [ ]:
feat_builder = st.FeatsFromOnlyEmpath()
empath_corpus = st.CorpusFromParsedDocuments(airlinesQnR,
                                              category_col='author_id_y',
                                              feats_from_spacy_doc=feat_builder,
                                              parsed_col='parsed').build()

In [ ]:
html = st.produce_scattertext_explorer(empath_corpus,
                                        category='British_Airways',
                                        category_name='British Airways',
                                        not_category_name='American Airlines',
                                        width_in_pixels=700,
                                        metadata=airlinesQnR['author_id_y'],
                                        use_non_text_features=True,
                                        use_full_doc=True,
                                        topic_model_term_lists=feat_builder.get_top_model_term_lists())


In [ ]:
filename = "empath-BA-vs-AA.html"
open(filename, 'wb').write(html.encode('utf-8'))
IFrame(src=filename, width = 900, height=700)

In [ ]:
corpus = (st.CorpusFromParsedDocuments(airlinesQnR,
                             category_col='author_id_y',
                             parsed_col='parsed').build().get_stoplisted_unigram_corpus())


In [ ]:
target_term = 'delay'
html = st.word_similarity_explorer(corpus,
                                   category='British_Airways',
                                   category_name='British Airways',
                                   not_category_name='American Airlines',
                                   target_term=target_term,
                                   minimum_term_frequency=5,
                                   width_in_pixels=800)


In [ ]:
file_name = 'similarity.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1000, height=700)

In [ ]:
html = st.produce_projection_explorer(corpus,
                                   category='British_Airways',
                                   category_name='British Airways',
                                   not_category_name='American Airlines',
                                   width_in_pixels=800)

**Lower case**

In [ ]:
def remove_uppercase(text):
    text_lowercase = ' '.join(x.lower() for x in text.split())# It will discard all uppercases
    return text_lowercase

In [ ]:
QnR['text_x_clean'] = QnR['text_x'].apply(lambda x: remove_uppercase(x))
QnR['text_y_clean'] = QnR['text_y'].apply(lambda x: remove_uppercase(x))
#in modo da poter rimuovere i nomi delle compagnie
QnR['author_id_y'] = QnR['author_id_y'].apply(lambda x: remove_uppercase(x))

In [ ]:
QnR.head(3)

**Remove Puntuaction**


In [ ]:
import string
string.punctuation

In [ ]:
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])# It will discard all punctuations
    return text_nopunct

In [ ]:
QnR['text_x_clean'] = QnR['text_x_clean'].apply(lambda x: remove_punct(x))
QnR['text_y_clean'] = QnR['text_y_clean'].apply(lambda x: remove_punct(x))
QnR.head(3)

**Removing usernames**

In [ ]:
# usernames = QnR['author_id_x'].unique()
companies = QnR['author_id_y'].unique()

In [ ]:
# gli username dei clienti sono numeri
QnR['text_x_clean'] = QnR['text_x_clean'].str.replace('\d+', '')
QnR['text_y_clean'] = QnR['text_y_clean'].str.replace('\d+', '')

In [ ]:
QnR['text_x_clean'] = QnR['text_x_clean'].str.replace('|'.join(companies), '')
QnR['text_y_clean'] = QnR['text_y_clean'].str.replace('|'.join(companies), '')
QnR.head(3)

**Checking Most Common Words**

In [ ]:
freqX = pd.Series(' '.join(QnR['text_x_clean']).split()).value_counts()[:10]
freqY = pd.Series(' '.join(QnR['text_y_clean']).split()).value_counts()[:10]
print('FREQ X: \n',freqX,'\nFREQ Y: \n', freqY)

In [ ]:
#removing them
freqX = list(freqX.index)
freqY = list(freqY.index)
QnR['text_x_clean'] = QnR['text_x_clean'].apply(lambda x: " ".join(x for x in x.split() if x not in freqX))
QnR['text_y_clean'] = QnR['text_y_clean'].apply(lambda x: " ".join(x for x in x.split() if x not in freqY))

**Checking Most Rare Words**

In [ ]:
rareX = pd.Series(' '.join(QnR['text_x_clean']).split()).value_counts()[-100:]
rareY = pd.Series(' '.join(QnR['text_y_clean']).split()).value_counts()[-100:]
print('RARE X: \n',rareX,'\nRARE Y: \n', rareY)

In [ ]:
#removing them
rareX = list(rareX.index)
rareY = list(rareY.index)
QnR['text_x_clean'] = QnR['text_x_clean'].apply(lambda x: " ".join(x for x in x.split() if x not in rareX))
QnR['text_y_clean'] = QnR['text_y_clean'].apply(lambda x: " ".join(x for x in x.split() if x not in rareY))

**Tokenizing**

In [ ]:
import re

# Function to Tokenize words
def tokenize(text):
    tokens = re.split('\W+', text) #W+ means that either a word character (A-Za-z0-9_) or a dash (-) can go there.
    return tokens

In [ ]:
QnR['text_x_tokenized'] = QnR['text_x_clean'].apply(lambda x: tokenize(x.lower())) 
QnR['text_y_tokenized'] = QnR['text_y_clean'].apply(lambda x: tokenize(x.lower()))
#We convert to lower as Python is case-sensitive.
QnR.head(3)

**Remove StopWords**

In [ ]:
import nltk

stopword = nltk.corpus.stopwords.words('english') # All English Stopwords

# Function to remove Stopwords
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]# To remove all stopwords
    return text

QnR['text_x_tokenized'] = QnR['text_x_tokenized'].apply(lambda x: remove_stopwords(x))
QnR['text_y_tokenized'] = QnR['text_y_tokenized'].apply(lambda x: remove_stopwords(x))
QnR.head(3)

**Lemmatizing**

In [ ]:
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()

def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

In [ ]:
QnR['text_x_lemmatized'] = QnR['text_x_tokenized'].apply(lambda x: lemmatizing(x))
QnR['text_y_lemmatized'] = QnR['text_y_tokenized'].apply(lambda x: lemmatizing(x))
QnR.head(3)

**Count Vectorizer**

In [ ]:
## Load the library with the CountVectorizer method
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
questions = QnR['text_x_clean'].dropna()
q = np.array(questions)

In [ ]:
countV = CountVectorizer(stop_words='english',
                         max_features=10000)

# Fit and transform the processed titles
bagQuestions = countV.fit_transform(q)

print('BOW Questions: ',bagQuestions.shape)

In [ ]:
words = countV.get_feature_names()
total_counts = np.zeros(len(words))
for t in bagQuestions:
    total_counts += t.toarray()[0]
    
count_dict = (zip(words, total_counts))
count_dict = sorted(count_dict, key=lambda x:x[1], reverse=True)
d = dict(count_dict)

In [ ]:
from wordcloud import WordCloud

sns.set_style('white')
plt.figure(figsize=(15,15))
wc = WordCloud(background_color="white",width=1000,height=1000, max_words=50,
               relative_scaling=0.5,normalize_plurals=False).generate_from_frequencies(d)
plt.title('WordCloud')
plt.imshow(wc)
plt.show()

## **Seq2Seq  MODEL DEVELOPMENT**

In [ ]:
from nltk.tokenize import casual_tokenize
count_vec = CountVectorizer(tokenizer=casual_tokenize, max_features=MAX_VOCAB_SIZE - 3)
print("Fitting CountVectorizer on X and Y text data...")
count_vec.fit(tqdm(QnR["text_x"] + QnR["text_y"]))
analyzer = count_vec.build_analyzer()
vocab = {k: v + 3 for k, v in count_vec.vocabulary_.items()}
vocab['__unk__'] = UNK
vocab['__pad__'] = PAD
vocab['__start__'] = START
# Used to turn seq2seq predictions into human readable strings
reverse_vocab = {v: k for k, v in vocab.items()}
print(f"Learned vocab of {len(vocab)} items.")

In [ ]:
def to_word_idx(sentence):
    full_length = [vocab.get(tok, UNK) for tok in analyzer(sentence)] + [PAD] * MAX_MESSAGE_LEN
    return full_length[:MAX_MESSAGE_LEN]

def from_word_idx(word_idxs):
    return ' '.join(reverse_vocab[idx] for idx in word_idxs if idx != PAD).strip()

In [ ]:
# Make sure our helpers work as expected...
QnR["text_x"].head().apply(to_word_idx).apply(from_word_idx)

In [ ]:
print("Calculating word indexes for X...")
x = pd.np.vstack(QnR["text_x"].progress_apply(to_word_idx).values)
print("Calculating word indexes for Y...")
y = pd.np.vstack(QnR["text_y"].progress_apply(to_word_idx).values)

In [ ]:
all_idx = list(range(len(x)))
train_idx = set(random.sample(all_idx, int(0.8 * len(all_idx))))
test_idx = {idx for idx in all_idx if idx not in train_idx}

train_x = x[list(train_idx)]
test_x = x[list(test_idx)]
train_y = y[list(train_idx)]
test_y = y[list(test_idx)]

assert train_x.shape == train_y.shape
assert test_x.shape == test_y.shape

print(f'Training data of shape {train_x.shape} and test data of shape {test_x.shape}.')

In [ ]:
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense, Input, LSTM, Dropout, Embedding, RepeatVector, concatenate, TimeDistributed
from keras.utils import np_utils

In [ ]:
def create_model():
    shared_embedding = Embedding(
        output_dim=EMBEDDING_SIZE,
        input_dim=MAX_VOCAB_SIZE,
        input_length=MAX_MESSAGE_LEN,
        name='embedding',
    )
    
    # ENCODER
    
    encoder_input = Input(
        shape=(MAX_MESSAGE_LEN,),
        dtype='int32',
        name='encoder_input',
    )
    
    embedded_input = shared_embedding(encoder_input)
    
    # No return_sequences - since the encoder here only produces a single value for the
    # input sequence provided.
    encoder_rnn = LSTM(
        CONTEXT_SIZE,
        name='encoder',
        dropout=DROPOUT
    )
    
    context = RepeatVector(MAX_MESSAGE_LEN)(encoder_rnn(embedded_input))
    
    # DECODER
    
    last_word_input = Input(
        shape=(MAX_MESSAGE_LEN, ),
        dtype='int32',
        name='last_word_input',
    )
    
    embedded_last_word = shared_embedding(last_word_input)
    # Combines the context produced by the encoder and the last word uttered as inputs
    # to the decoder.
    decoder_input = concatenate([embedded_last_word, context], axis=2)
    
    # return_sequences causes LSTM to produce one output per timestep instead of one at the
    # end of the intput, which is important for sequence producing models.
    decoder_rnn = LSTM(
        CONTEXT_SIZE,
        name='decoder',
        return_sequences=True,
        dropout=DROPOUT
    )
    
    decoder_output = decoder_rnn(decoder_input)
    
    # TimeDistributed allows the dense layer to be applied to each decoder output per timestep
    next_word_dense = TimeDistributed(
        Dense(int(MAX_VOCAB_SIZE / 2), activation='relu'),
        name='next_word_dense',
    )(decoder_output)
    
    next_word = TimeDistributed(
        Dense(MAX_VOCAB_SIZE, activation='softmax'),
        name='next_word_softmax'
    )(next_word_dense)
    
    return Model(inputs=[encoder_input, last_word_input], outputs=[next_word])

s2s_model = create_model()
optimizer = Adam(lr=LEARNING_RATE, clipvalue=5.0)
s2s_model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
def add_start_token(y_array):
    """ Adds the start token to vectors.  Used for training data. """
    return np.hstack([
        START * np.ones((len(y_array), 1)),
        y_array[:, :-1],
    ])

def binarize_labels(labels):
    """ Helper function that turns integer word indexes into sparse binary matrices for 
        the expected model output.
    """
    return np.array([np_utils.to_categorical(row, num_classes=MAX_VOCAB_SIZE)
                     for row in labels])

In [ ]:
def respond_to(model, text):
    """ Helper function that takes a text input and provides a text output. """
    input_y = add_start_token(PAD * np.ones((1, MAX_MESSAGE_LEN)))
    idxs = np.array(to_word_idx(text)).reshape((1, MAX_MESSAGE_LEN))
    for position in range(MAX_MESSAGE_LEN - 1):
        prediction = model.predict([idxs, input_y]).argmax(axis=2)[0]
        input_y[:,position + 1] = prediction[position]
    return from_word_idx(model.predict([idxs, input_y]).argmax(axis=2)[0])

In [ ]:
def train_mini_epoch(model, start_idx, end_idx):
    """ Batching seems necessary in Kaggle Jupyter Notebook environments, since
        `model.fit` seems to freeze on larger batches (somewhere 1k-10k).
    """
    b_train_y = binarize_labels(train_y[start_idx:end_idx])
    input_train_y = add_start_token(train_y[start_idx:end_idx])
    
    model.fit(
        [train_x[start_idx:end_idx], input_train_y], 
        b_train_y,
        epochs=1,
        batch_size=BATCH_SIZE,
    )
    
    rand_idx = random.sample(list(range(len(test_x))), SUB_BATCH_SIZE)
    print('Test results:', model.evaluate(
        [test_x[rand_idx], add_start_token(test_y[rand_idx])],
        binarize_labels(test_y[rand_idx])
    ))
    
    input_strings = [
        "@AppleSupport I fix I this I stupid I problem I",
        "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service.",
    ]
    
    for input_string in input_strings:
        output_string = respond_to(model, input_string)
        print(f'> "{input_string}"\n< "{output_string}"')

In [ ]:
training_time_limit = 360 * 60  # seconds (notebooks terminate after 1 hour)
start_time = time.time()
stop_after = start_time + training_time_limit

class TimesUpInterrupt(Exception):
    pass

try:
    for epoch in range(100):
        print(f'Training in epoch {epoch}...')
        for start_idx in range(0, len(train_x), SUB_BATCH_SIZE):
            train_mini_epoch(s2s_model, start_idx, start_idx + SUB_BATCH_SIZE)
            if time.time() > stop_after:
                raise TimesUpInterrupt
except KeyboardInterrupt:
    print("Halting training from keyboard interrupt.")
except TimesUpInterrupt:
    print(f"Halting after {time.time() - start_time} seconds spent training.")

In [ ]:
respond_to(s2s_model, '''@AppleSupport iPhone 8 touchID doesnt unlock while charging on 
    110v w/ 61w laptop charger to usbc lightning cable just uh.. so you guys know''')

In [ ]:
respond_to(s2s_model, '''@sprintcare I can't make calls... wtf''')